# PSO 02

### Instructor: Huangyi Ge

- PSO 01/05 materials: [github](https://github.com/gehuangyi20/cs448)

---

Serve html slides

`jupyter nbconvert 2020-02-01\ PSO\ 02.ipynb.ipynb --to slides --post serve`

Make markdown file:

`jupyter nbconvert 2020-02-01\ PSO\ 02.ipynb  --to markdown  --TagRemovePreprocessor.remove_cell_tags="remove_cell"`

---

In [1]:
%load_ext sql

In [2]:
import pandas as pd

In [3]:
import tabulate

In [4]:
%%bash 
cp resources/sql-original.db sql.db

In [5]:
%sql sqlite:///resources/sql.db

### Setup

1. [Download database](resources/sql-original.db)
2. Start SQLite `$sqlite3`
3. Open database `sqlite> .open sql-original.db`
4. Setup headers
```
sqlite> .header on
sqlite> .mode column
```

### Schema for instructor relation

Recall the schema that we used previously:

![Figure 2.9: Schema diagram for instructor relation](figures/ERD-instructor.png)

#### SQL data definition

```sql
create table instructor
    (ID             varchar(5), 
     name           varchar(20) not null, 
     dept_name      varchar(20), 
     salary         numeric(8,2) check (salary > 29000),
     primary key (ID),
     foreign key (dept_name) references department (dept_name)
        on delete set null
    );
```

In [6]:
result = %sql select * from instructor;
result = result.DataFrame()
print(result.to_markdown(index=False))

 * sqlite:///resources/sql.db
Done.
|    ID | name       | dept_name   |   salary |
|------:|:-----------|:------------|---------:|
| 10101 | Srinivasan | Comp. Sci.  |    65000 |
| 12121 | Wu         | Finance     |    90000 |
| 15151 | Mozart     | Music       |    40000 |
| 22222 | Einstein   | Physics     |    95000 |
| 32343 | El Said    | History     |    60000 |
| 33456 | Gold       | Physics     |    87000 |
| 45565 | Katz       | Comp. Sci.  |    75000 |
| 58583 | Califieri  | History     |    62000 |
| 76543 | Singh      | Finance     |    80000 |
| 76766 | Crick      | Biology     |    72000 |
| 83821 | Brandt     | Comp. Sci.  |    92000 |
| 98345 | Kim        | Elec. Eng.  |    80000 |


#### Example data

|    ID | name       | dept_name   |   salary |
|------:|:-----------|:------------|---------:|
| 10101 | Srinivasan | Comp. Sci.  |    65000 |
| 12121 | Wu         | Finance     |    90000 |
| 15151 | Mozart     | Music       |    40000 |
| 22222 | Einstein   | Physics     |    95000 |
| 32343 | El Said    | History     |    60000 |
| 33456 | Gold       | Physics     |    87000 |
| 45565 | Katz       | Comp. Sci.  |    75000 |
| 58583 | Califieri  | History     |    62000 |
| 76543 | Singh      | Finance     |    80000 |
| 76766 | Crick      | Biology     |    72000 |
| 83821 | Brandt     | Comp. Sci.  |    92000 |
| 98345 | Kim        | Elec. Eng.  |    80000 |


## Schema Evolution

- Database design is usually not a one-time activity. 
- The needs of an organization evolve continually, and the data that it needs to store also evolve correspondingly.
- Changes in the schema can affect all aspects of the database application.

## Schema for instructor relation

Lets say that we want to add a phone number for instructors.

### Option 1

We could simply add an attribute *phone_number* to the _instructor_ relation.

![Option 1: Schema diagram for instructor relation](figures/ERD-instructor-Option_1.png)


#### ALTER TABLE Command

In SQLite, we add a new column `phone_number` to the `instructor` table as follows:

```sql
ALTER TABLE instructor ADD COLUMN phone_number varchar(12);
```

In [7]:
%sql ALTER TABLE instructor ADD COLUMN phone_number varchar(12);

 * sqlite:///resources/sql.db
Done.


[]

In [8]:
import random
def random_phone_num_generator():
    first = str(random.randint(100, 999))
    second = str(random.randint(1, 888)).zfill(3)

    last = (str(random.randint(1, 9998)).zfill(4))
    while last in ['1111', '2222', '3333', '4444', '5555', '6666', '7777', '8888']:
        last = (str(random.randint(1, 9998)).zfill(4))

    return '{}-{}-{}'.format(first, second, last)

In [9]:
result = %sql select ID from instructor;
num_instructors = len(result)

gen_phone_numbers = [random_phone_num_generator() for i in range(num_instructors)]
assert len(gen_phone_numbers) == len(set(gen_phone_numbers))
update_commands = []

for idx, record in enumerate(result[:-1]):
    uid = record[0]
    update = f"UPDATE instructor SET phone_number = '{gen_phone_numbers[idx]}' WHERE ID = {uid};"
    update_commands.append(update)
    print(update)

 * sqlite:///resources/sql.db
Done.
UPDATE instructor SET phone_number = '358-777-1265' WHERE ID = 10101;
UPDATE instructor SET phone_number = '492-554-3421' WHERE ID = 12121;
UPDATE instructor SET phone_number = '722-035-7216' WHERE ID = 15151;
UPDATE instructor SET phone_number = '275-026-7979' WHERE ID = 22222;
UPDATE instructor SET phone_number = '175-304-5605' WHERE ID = 32343;
UPDATE instructor SET phone_number = '132-360-4775' WHERE ID = 33456;
UPDATE instructor SET phone_number = '182-846-5531' WHERE ID = 45565;
UPDATE instructor SET phone_number = '722-419-0625' WHERE ID = 58583;
UPDATE instructor SET phone_number = '802-567-3477' WHERE ID = 76543;
UPDATE instructor SET phone_number = '735-031-4651' WHERE ID = 76766;
UPDATE instructor SET phone_number = '573-255-8741' WHERE ID = 83821;


#### Update Records

Then, we can add phone numbers for each instructor using `UPDATE`

```sql
UPDATE instructor SET phone_number = '358-777-1265' WHERE ID = 10101;
UPDATE instructor SET phone_number = '492-554-3421' WHERE ID = 12121;
UPDATE instructor SET phone_number = '722-035-7216' WHERE ID = 15151;
UPDATE instructor SET phone_number = '275-026-7979' WHERE ID = 22222;
UPDATE instructor SET phone_number = '175-304-5605' WHERE ID = 32343;
UPDATE instructor SET phone_number = '132-360-4775' WHERE ID = 33456;
UPDATE instructor SET phone_number = '182-846-5531' WHERE ID = 45565;
UPDATE instructor SET phone_number = '722-419-0625' WHERE ID = 58583;
UPDATE instructor SET phone_number = '802-567-3477' WHERE ID = 76543;
UPDATE instructor SET phone_number = '735-031-4651' WHERE ID = 76766;
UPDATE instructor SET phone_number = '573-255-8741' WHERE ID = 83821;
```

Note: we haven't added a phone number for instructor Kim (ID = 98345).

In [10]:
for update in update_commands:
    %sql $update

 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.
 * sqlite:///resources/sql.db
1 rows affected.


In [11]:
result = %sql select * from instructor;
result = result.DataFrame()
print(result.to_markdown(index=False))

 * sqlite:///resources/sql.db
Done.
|    ID | name       | dept_name   |   salary | phone_number   |
|------:|:-----------|:------------|---------:|:---------------|
| 10101 | Srinivasan | Comp. Sci.  |    65000 | 358-777-1265   |
| 12121 | Wu         | Finance     |    90000 | 492-554-3421   |
| 15151 | Mozart     | Music       |    40000 | 722-035-7216   |
| 22222 | Einstein   | Physics     |    95000 | 275-026-7979   |
| 32343 | El Said    | History     |    60000 | 175-304-5605   |
| 33456 | Gold       | Physics     |    87000 | 132-360-4775   |
| 45565 | Katz       | Comp. Sci.  |    75000 | 182-846-5531   |
| 58583 | Califieri  | History     |    62000 | 722-419-0625   |
| 76543 | Singh      | Finance     |    80000 | 802-567-3477   |
| 76766 | Crick      | Biology     |    72000 | 735-031-4651   |
| 83821 | Brandt     | Comp. Sci.  |    92000 | 573-255-8741   |
| 98345 | Kim        | Elec. Eng.  |    80000 |                |


#### Updated Table

We can take a look at the `instructor` table with `phone_number` column and values we inserted:

```sql
select * from instructor;
```

|    ID | name       | dept_name   |   salary | phone_number   |
|------:|:-----------|:------------|---------:|:---------------|
| 10101 | Srinivasan | Comp. Sci.  |    65000 | 358-777-1265   |
| 12121 | Wu         | Finance     |    90000 | 492-554-3421   |
| 15151 | Mozart     | Music       |    40000 | 722-035-7216   |
| 22222 | Einstein   | Physics     |    95000 | 275-026-7979   |
| 32343 | El Said    | History     |    60000 | 175-304-5605   |
| 33456 | Gold       | Physics     |    87000 | 132-360-4775   |
| 45565 | Katz       | Comp. Sci.  |    75000 | 182-846-5531   |
| 58583 | Califieri  | History     |    62000 | 722-419-0625   |
| 76543 | Singh      | Finance     |    80000 | 802-567-3477   |
| 76766 | Crick      | Biology     |    72000 | 735-031-4651   |
| 83821 | Brandt     | Comp. Sci.  |    92000 | 573-255-8741   |
| 98345 | Kim        | Elec. Eng.  |    80000 |                |

Remember: we didn't add a phone number for instructor Kim (ID = 98345).

## Schema for instructor relation

What if we want to have multiple phone number for instructors?

### Option 2

If we make the attribute `phone_number` part of the primary key, then we could have multiple phone numbers per instructor.

![Option 2: Schema diagram for instructor relation](figures/ERD-instructor-Option_2.png)

#### Example data

|    ID | name       | dept_name   |   salary | phone_number   |
|------:|:-----------|:------------|---------:|:---------------|
| 10101 | Srinivasan | Comp. Sci.  |    65000 | 999-113-1268   |
| 10101 | Srinivasan | Comp. Sci.  |    80000 | 906-214-7631   |

What's wrong with this approach?

- It creates redundancy in the database (breaking Boyce-Codd Normal Form).
- It can cause data inconsistencies (Ex. updated salary for an instructor for only 1 record).

## Schema for instructor relation

What if we want to have multiple phone number for instructors?

### Option 3

Use of phone as an entity allows extra information about phone numbers (plus multiple phone numbers).

![Option 3: Schema diagram for instructor relation](figures/ERD-instructor-Option_3.png)

##### SQLite ALTER TABLE

- SQLite supports a limited subset of `ALTER TABLE`. 
- The `ALTER TABLE` command in SQLite allows the user to rename a table or to add a new column to an existing table. 
- It is **not possible** to:
  - _rename_ or _remove_ a column, or 
  - _add_ or _remove_ constraints from a table.

Therefore, the following will **not work** on SQLite:

```sql
ALTER TABLE instructor DROP COLUMN phone_number;
```

To "undo" the changes from Option (a), we can do the following:

1. create new table without the `phone_number` column,
2. copy all records,
3. drop the old table,
4. rename the new table.

#### 1. Create new instructor table

```sql
CREATE TABLE instructor_new
    (ID             varchar(5), 
     name           varchar(20) NOT NULL, 
     dept_name      varchar(20), 
     salary         numeric(8,2) check (salary > 29000),
     PRIMARY KEY (ID),
     FOREIGN KEY (dept_name) REFERENCES department (dept_name)
        ON DELETE SET NULL
    );
```

In [12]:
%%sql
CREATE TABLE instructor_new
    (ID             varchar(5), 
     name           varchar(20) NOT NULL, 
     dept_name      varchar(20), 
     salary         numeric(8,2) check (salary > 29000),
     PRIMARY KEY (ID),
     FOREIGN KEY (dept_name) REFERENCES department (dept_name)
        ON DELETE SET NULL
    );

 * sqlite:///resources/sql.db
Done.


[]

#### 2. Copy all records

```sql
INSERT INTO instructor_new
SELECT instructor.ID, instructor.name, instructor.dept_name, instructor.salary
FROM instructor;
```

In [13]:
%%sql 
INSERT INTO instructor_new
SELECT instructor.ID, instructor.name, instructor.dept_name, instructor.salary
FROM instructor;

 * sqlite:///resources/sql.db
12 rows affected.


[]

#### 3. Drop old table

```sql
DROP TABLE instructor;
```

In [14]:
%sql DROP TABLE instructor;

 * sqlite:///resources/sql.db
Done.


[]

#### 4. Rename new table

```sql
ALTER TABLE instructor_new RENAME TO instructor;
```

In [15]:
%sql ALTER TABLE instructor_new RENAME TO instructor;

 * sqlite:///resources/sql.db
Done.


1


## Schema for instructor relation

What if we want to have multiple phone number for instructors?

### Option 3

Use of phone as an entity allows extra information about phone numbers (plus multiple phone numbers).

![Option 3: Schema diagram for instructor relation](figures/ERD-instructor-Option_3.png)

#### SQL data definitions

Create a table to store and assign the phone numbers:

```sql
CREATE TABLE phone
    (phone_number   varchar(12), 
     location       varchar(6)
             CHECK (location IN ('Office', 'Cell', 'Home', 'Other')), 
     ID             varchar(5), 
     PRIMARY KEY (phone_number),
     FOREIGN KEY (ID) REFERENCES instructor (ID)
            ON DELETE SET NULL
    );
```


#### Questions

1. Does this require that everyone has a phone number?
3. What happens if we make the instructor ID part of the primary key?

#### Answers

1. This schema does not require instructors to have a phone number.
3. Making the instructor ID part of the primary key allows for instructors to have the same phone number.

In [18]:
%sql DROP TABLE phone;

 * sqlite:///resources/sql.db
Done.


[]

In [19]:
%%sql

CREATE TABLE phone
    (phone_number   varchar(12), 
     location       varchar(6)
             CHECK (location IN ('Office', 'Cell', 'Home', 'Other')), 
     ID             varchar(5), 
     PRIMARY KEY (phone_number),
     FOREIGN KEY (ID) REFERENCES instructor (ID)
            ON DELETE SET NULL
    );

 * sqlite:///resources/sql.db
Done.


[]

In [20]:
result = %sql select ID from instructor;
num_instructors = len(result)

alt_phone_numbers = [random_phone_num_generator() for i in range(num_instructors)]
assert len(alt_phone_numbers) == len(set(alt_phone_numbers))
locations = {'Office': gen_phone_numbers, 'Cell': alt_phone_numbers}

phone_commands = []
phone_inst_commands = []
for location in ['Office', 'Cell']:
    for idx, record in enumerate(result[:-1]):
        uid = record[0]
        phone = locations[location][idx]
        
        # phone table
        insert = f"INSERT INTO phone VALUES ('{phone}', '{location}', '{uid}');"
        phone_commands.append(insert)

 * sqlite:///resources/sql.db
Done.


In [21]:
for insert in phone_commands:
    print(insert)

for insert in phone_commands:
    %sql $insert

INSERT INTO phone VALUES ('358-777-1265', 'Office', '10101');
INSERT INTO phone VALUES ('492-554-3421', 'Office', '12121');
INSERT INTO phone VALUES ('722-035-7216', 'Office', '15151');
INSERT INTO phone VALUES ('275-026-7979', 'Office', '22222');
INSERT INTO phone VALUES ('175-304-5605', 'Office', '32343');
INSERT INTO phone VALUES ('132-360-4775', 'Office', '33456');
INSERT INTO phone VALUES ('182-846-5531', 'Office', '45565');
INSERT INTO phone VALUES ('722-419-0625', 'Office', '58583');
INSERT INTO phone VALUES ('802-567-3477', 'Office', '76543');
INSERT INTO phone VALUES ('735-031-4651', 'Office', '76766');
INSERT INTO phone VALUES ('573-255-8741', 'Office', '83821');
INSERT INTO phone VALUES ('711-239-7079', 'Cell', '10101');
INSERT INTO phone VALUES ('357-271-7033', 'Cell', '12121');
INSERT INTO phone VALUES ('865-740-1566', 'Cell', '15151');
INSERT INTO phone VALUES ('341-066-0914', 'Cell', '22222');
INSERT INTO phone VALUES ('843-858-5992', 'Cell', '32343');
INSERT INTO phone 

#### INSERT Commands for Table: phone

These will add the phone numbers:

```sql
INSERT INTO phone VALUES ('358-777-1265', 'Office', '10101');
INSERT INTO phone VALUES ('492-554-3421', 'Office', '12121');
INSERT INTO phone VALUES ('722-035-7216', 'Office', '15151');
INSERT INTO phone VALUES ('275-026-7979', 'Office', '22222');
INSERT INTO phone VALUES ('175-304-5605', 'Office', '32343');
INSERT INTO phone VALUES ('132-360-4775', 'Office', '33456');
INSERT INTO phone VALUES ('182-846-5531', 'Office', '45565');
INSERT INTO phone VALUES ('722-419-0625', 'Office', '58583');
INSERT INTO phone VALUES ('802-567-3477', 'Office', '76543');
INSERT INTO phone VALUES ('735-031-4651', 'Office', '76766');
INSERT INTO phone VALUES ('573-255-8741', 'Office', '83821');
INSERT INTO phone VALUES ('711-239-7079', 'Cell', '10101');
INSERT INTO phone VALUES ('357-271-7033', 'Cell', '12121');
INSERT INTO phone VALUES ('865-740-1566', 'Cell', '15151');
INSERT INTO phone VALUES ('341-066-0914', 'Cell', '22222');
INSERT INTO phone VALUES ('843-858-5992', 'Cell', '32343');
INSERT INTO phone VALUES ('386-110-2746', 'Cell', '33456');
INSERT INTO phone VALUES ('532-340-6156', 'Cell', '45565');
INSERT INTO phone VALUES ('593-208-3674', 'Cell', '58583');
INSERT INTO phone VALUES ('851-114-9021', 'Cell', '76543');
INSERT INTO phone VALUES ('759-309-1635', 'Cell', '76766');
INSERT INTO phone VALUES ('691-011-8290', 'Cell', '83821');
```

Note: we haven't added a phone numbers for instructor Kim (ID = 98345).

### Updated records

Show the instructor IDs, names, and _Office_ phone numbers

#### Query

```sql
SELECT instructor.ID, instructor.name, phone.phone_number, phone.location 
FROM instructor
LEFT JOIN phone ON instructor.ID = phone.ID
WHERE location = 'Office';
```

In [22]:
query = """SELECT instructor.ID, instructor.name, phone.phone_number, phone.location 
FROM instructor
LEFT JOIN phone ON instructor.ID = phone.ID
WHERE location = 'Office';"""
query_res = %sql $query
query_res = query_res.DataFrame()
print(query_res.to_markdown(index=False))

 * sqlite:///resources/sql.db
Done.
|    ID | name       | phone_number   | location   |
|------:|:-----------|:---------------|:-----------|
| 10101 | Srinivasan | 358-777-1265   | Office     |
| 12121 | Wu         | 492-554-3421   | Office     |
| 15151 | Mozart     | 722-035-7216   | Office     |
| 22222 | Einstein   | 275-026-7979   | Office     |
| 32343 | El Said    | 175-304-5605   | Office     |
| 33456 | Gold       | 132-360-4775   | Office     |
| 45565 | Katz       | 182-846-5531   | Office     |
| 58583 | Califieri  | 722-419-0625   | Office     |
| 76543 | Singh      | 802-567-3477   | Office     |
| 76766 | Crick      | 735-031-4651   | Office     |
| 83821 | Brandt     | 573-255-8741   | Office     |


#### Result

|    ID | name       | phone_number   | location   |
|------:|:-----------|:---------------|:-----------|
| 10101 | Srinivasan | 358-777-1265   | Office     |
| 12121 | Wu         | 492-554-3421   | Office     |
| 15151 | Mozart     | 722-035-7216   | Office     |
| 22222 | Einstein   | 275-026-7979   | Office     |
| 32343 | El Said    | 175-304-5605   | Office     |
| 33456 | Gold       | 132-360-4775   | Office     |
| 45565 | Katz       | 182-846-5531   | Office     |
| 58583 | Califieri  | 722-419-0625   | Office     |
| 76543 | Singh      | 802-567-3477   | Office     |
| 76766 | Crick      | 735-031-4651   | Office     |
| 83821 | Brandt     | 573-255-8741   | Office     |

Note: There is no phone number for instructor Kim (ID = 98345), so their record does not show because we are filtering by phone location.

### Updated records

Show the instructor IDs, names, and phone numbers for _Office_ (if any).

#### Query

```sql
SELECT instructor.ID, instructor.name, phone.phone_number, phone.location 
FROM instructor
LEFT JOIN phone ON instructor.ID = phone.ID
WHERE location = 'Office' OR location IS NULL;
```

In [27]:
query = """SELECT instructor.ID, instructor.name, phone.phone_number, phone.location 
FROM instructor
LEFT JOIN phone ON instructor.ID = phone.ID
WHERE location = 'Office' OR location IS NULL;"""
query_res = %sql $query
query_res = query_res.DataFrame()
print(query_res.to_markdown(index=False))

 * sqlite:///resources/sql.db
Done.
|    ID | name       | phone_number   | location   |
|------:|:-----------|:---------------|:-----------|
| 10101 | Srinivasan | 358-777-1265   | Office     |
| 12121 | Wu         | 492-554-3421   | Office     |
| 15151 | Mozart     | 722-035-7216   | Office     |
| 22222 | Einstein   | 275-026-7979   | Office     |
| 32343 | El Said    | 175-304-5605   | Office     |
| 33456 | Gold       | 132-360-4775   | Office     |
| 45565 | Katz       | 182-846-5531   | Office     |
| 58583 | Califieri  | 722-419-0625   | Office     |
| 76543 | Singh      | 802-567-3477   | Office     |
| 76766 | Crick      | 735-031-4651   | Office     |
| 83821 | Brandt     | 573-255-8741   | Office     |
| 98345 | Kim        |                |            |


#### Result

|    ID | name       | phone_number   | location   |
|------:|:-----------|:---------------|:-----------|
| 10101 | Srinivasan | 358-777-1265   | Office     |
| 12121 | Wu         | 492-554-3421   | Office     |
| 15151 | Mozart     | 722-035-7216   | Office     |
| 22222 | Einstein   | 275-026-7979   | Office     |
| 32343 | El Said    | 175-304-5605   | Office     |
| 33456 | Gold       | 132-360-4775   | Office     |
| 45565 | Katz       | 182-846-5531   | Office     |
| 58583 | Califieri  | 722-419-0625   | Office     |
| 76543 | Singh      | 802-567-3477   | Office     |
| 76766 | Crick      | 735-031-4651   | Office     |
| 83821 | Brandt     | 573-255-8741   | Office     |
| 98345 | Kim        |                |            |

Note: There are no phone numbers for instructor Kim (ID = 98345) so the column appears empty.

---

## Schema for instructor relation

What if we want to have multiple phone number for instructors?

### Option 4

Use of phone as an entity allows extra information about phone numbers (plus multiple phone numbers).

![Figure 6.3(b): Schema diagram for instructor relation](figures/ERD-instructor-Option_4.png)

#### SQL data definitions

Create a tables to store and assign the phone numbers:

```sql
CREATE TABLE phone
    (phone_number   varchar(12), 
     location       varchar(6)
             CHECK (location IN ('Office', 'Cell', 'Home', 'Other')),
     PRIMARY KEY (phone_number)
    );
    
CREATE TABLE phone_inst
    (ID             varchar(5),
     phone_number   varchar(12), 
     PRIMARY KEY (ID, phone_number),
     FOREIGN KEY (ID) REFERENCES instructor
            ON DELETE CASCADE
    );
```


In [ ]:
%%sql

CREATE TABLE phone
    (phone_number   varchar(12), 
     location       varchar(6)
             CHECK (location IN ('Office', 'Cell', 'Home', 'Other')),
     PRIMARY KEY (phone_number)
    );
    
CREATE TABLE phone_inst
    (ID             varchar(5),
     phone_number   varchar(12), 
     PRIMARY KEY (ID, phone_number),
     FOREIGN KEY (ID) REFERENCES instructor
            ON DELETE CASCADE
    );

In [ ]:
result = %sql select ID from instructor;
num_instructors = len(result)

alt_phone_numbers = [random_phone_num_generator() for i in range(num_instructors)]
assert len(alt_phone_numbers) == len(set(alt_phone_numbers))
locations = {'Office': gen_phone_numbers, 'Cell': alt_phone_numbers}

phone_commands = []
phone_inst_commands = []
for location in ['Office', 'Cell']:
    for idx, record in enumerate(result):
        uid = record[0]
        phone = locations[location][idx]
        
        # phone table
        insert = f"INSERT INTO phone VALUES ('{phone}', '{location}');"
        phone_commands.append(insert)
        
        # phone_inst table
        insert = f"INSERT INTO phone_inst VALUES ('{uid}', '{phone}');"
        phone_inst_commands.append(insert)

#### INSERT Commands for Table: phone

These will add the phone numbers:

```sql
INSERT INTO phone VALUES ('999-113-1268', 'Office');
INSERT INTO phone VALUES ('761-856-0718', 'Office');
INSERT INTO phone VALUES ('274-715-6841', 'Office');
INSERT INTO phone VALUES ('727-780-1695', 'Office');
INSERT INTO phone VALUES ('118-354-8109', 'Office');
INSERT INTO phone VALUES ('589-847-6616', 'Office');
INSERT INTO phone VALUES ('740-333-3446', 'Office');
INSERT INTO phone VALUES ('749-361-2350', 'Office');
INSERT INTO phone VALUES ('856-262-2788', 'Office');
INSERT INTO phone VALUES ('418-409-7638', 'Office');
INSERT INTO phone VALUES ('973-587-6940', 'Office');
INSERT INTO phone VALUES ('548-012-7539', 'Office');
INSERT INTO phone VALUES ('906-214-7631', 'Cell');
INSERT INTO phone VALUES ('851-591-5780', 'Cell');
INSERT INTO phone VALUES ('233-467-9285', 'Cell');
INSERT INTO phone VALUES ('318-795-0911', 'Cell');
INSERT INTO phone VALUES ('526-242-8559', 'Cell');
INSERT INTO phone VALUES ('488-016-4674', 'Cell');
INSERT INTO phone VALUES ('403-866-7060', 'Cell');
INSERT INTO phone VALUES ('540-653-4106', 'Cell');
INSERT INTO phone VALUES ('202-866-8444', 'Cell');
INSERT INTO phone VALUES ('407-517-2335', 'Cell');
INSERT INTO phone VALUES ('559-565-8598', 'Cell');
INSERT INTO phone VALUES ('804-869-1214', 'Cell');
```

In [ ]:
for insert in phone_commands:
    print(insert)

for insert in phone_commands:
    %sql $insert

#### INSERT Commands for Table: phone_inst

These will assign phone numbers to the instructors:

```sql
INSERT INTO phone_inst VALUES ('10101', '999-113-1268');
INSERT INTO phone_inst VALUES ('12121', '761-856-0718');
INSERT INTO phone_inst VALUES ('15151', '274-715-6841');
INSERT INTO phone_inst VALUES ('22222', '727-780-1695');
INSERT INTO phone_inst VALUES ('32343', '118-354-8109');
INSERT INTO phone_inst VALUES ('33456', '589-847-6616');
INSERT INTO phone_inst VALUES ('45565', '740-333-3446');
INSERT INTO phone_inst VALUES ('58583', '749-361-2350');
INSERT INTO phone_inst VALUES ('76543', '856-262-2788');
INSERT INTO phone_inst VALUES ('76766', '418-409-7638');
INSERT INTO phone_inst VALUES ('83821', '973-587-6940');
INSERT INTO phone_inst VALUES ('98345', '548-012-7539');
INSERT INTO phone_inst VALUES ('10101', '906-214-7631');
INSERT INTO phone_inst VALUES ('12121', '851-591-5780');
INSERT INTO phone_inst VALUES ('15151', '233-467-9285');
INSERT INTO phone_inst VALUES ('22222', '318-795-0911');
INSERT INTO phone_inst VALUES ('32343', '526-242-8559');
INSERT INTO phone_inst VALUES ('33456', '488-016-4674');
INSERT INTO phone_inst VALUES ('45565', '403-866-7060');
INSERT INTO phone_inst VALUES ('58583', '540-653-4106');
INSERT INTO phone_inst VALUES ('76543', '202-866-8444');
INSERT INTO phone_inst VALUES ('76766', '407-517-2335');
INSERT INTO phone_inst VALUES ('83821', '559-565-8598');
INSERT INTO phone_inst VALUES ('98345', '804-869-1214');
```

In [ ]:
for insert in phone_inst_commands:
    print(insert)
print()
for insert in phone_inst_commands:
    %sql $insert

In [ ]:
query = """SELECT instructor.ID, instructor.name, phone.phone_number, phone.location 
FROM instructor
LEFT JOIN phone_inst ON instructor.ID = phone_inst.ID
LEFT JOIN phone ON phone_inst.phone_number = phone.phone_number
WHERE location = 'Office';"""
query_res = %sql $query
query_res = query_res.DataFrame()
print(query_res.to_markdown(index=False))

### Updated records

Show the instructor IDs, names, and _Office_ phone numbers

#### Query

```sql
SELECT instructor.ID, instructor.name, phone.phone_number, phone.location 
FROM instructor
LEFT JOIN phone_inst ON instructor.ID = phone_inst.ID
LEFT JOIN phone ON phone_inst.phone_number = phone.phone_number
WHERE location = 'Office';
```

#### Result

|    ID | name       | phone_number   | location   |
|------:|:-----------|:---------------|:-----------|
| 10101 | Srinivasan | 999-113-1268   | Office     |
| 12121 | Wu         | 761-856-0718   | Office     |
| 15151 | Mozart     | 274-715-6841   | Office     |
| 22222 | Einstein   | 727-780-1695   | Office     |
| 32343 | El Said    | 118-354-8109   | Office     |
| 33456 | Gold       | 589-847-6616   | Office     |
| 45565 | Katz       | 740-333-3446   | Office     |
| 58583 | Califieri  | 749-361-2350   | Office     |
| 76543 | Singh      | 856-262-2788   | Office     |
| 76766 | Crick      | 418-409-7638   | Office     |
| 83821 | Brandt     | 973-587-6940   | Office     |
| 98345 | Kim        | 548-012-7539   | Office     |